In [46]:
import climt
from sympl import (
    PlotFunctionMonitor, NetCDFMonitor,
    TimeDifferencingWrapper, UpdateFrequencyWrapper,
)
import numpy as np
from datetime import timedelta
from gfs_dynamical_core import GFSDynamicalCore

In [47]:
def plot_function(fig, state):

    ax = fig.add_subplot(2, 2, 1)
    state['eastward_wind'].mean(dim='lon').plot.contourf(
        ax=ax, levels=16, robust=True)
    ax.set_title('Zonal Wind')

    ax = fig.add_subplot(2, 2, 2)
    state['northward_wind'].mean(dim='lon').plot.contourf(
        ax=ax, levels=16, robust=True)
    ax.set_title('Meridional Wind')

    ax = fig.add_subplot(2, 2, 3)
    state['air_temperature'].mean(dim='lon').plot.contourf(
        ax=ax, levels=16)
    ax.set_title('Temperature')

    ax = fig.add_subplot(2, 2, 4)
    state['specific_humidity'].mean(
        dim='lon').plot.contourf(
            ax=ax, levels=16, robust=True)
    ax.set_title('Specific Humidity')

    fig.tight_layout()

In [63]:
fields_to_store = ['air_temperature', 'air_pressure', 'eastward_wind',
                   'northward_wind', 'air_pressure_on_interface_levels',
                   'surface_pressure', 'specific_humidity', 'latitude', 
                   'longitude']

model_time_step = timedelta(seconds=600)
# Create components
dycore = GFSDynamicalCore(moist=False)
#grid = climt.get_grid(nx=128, ny=62)
grid = climt.get_grid(nx=64, ny=32, nz=13, latitude_grid='regular',
                        n_ice_interface_levels=None)

# Create model state
my_state = climt.get_default_state([dycore], grid_state=grid,
                                    n_ice_interface_levels=None)

# Create plotting object
monitor = PlotFunctionMonitor(plot_function)
netcdf_monitor = NetCDFMonitor('saving_test.nc',
                               write_on_store=True,
                               store_names=fields_to_store)

<Figure size 640x480 with 0 Axes>

In [64]:
# Set initial/boundary conditions
latitudes = my_state['latitude'].values
longitudes = my_state['longitude'].values

print("latitudes = \n", latitudes)
print("longitudes = \n", longitudes)

#surface_shape = [len(longitudes), len(latitudes)]

latitudes = 
 [[-87.1875 -87.1875 -87.1875 ... -87.1875 -87.1875 -87.1875]
 [-81.5625 -81.5625 -81.5625 ... -81.5625 -81.5625 -81.5625]
 [-75.9375 -75.9375 -75.9375 ... -75.9375 -75.9375 -75.9375]
 ...
 [ 75.9375  75.9375  75.9375 ...  75.9375  75.9375  75.9375]
 [ 81.5625  81.5625  81.5625 ...  81.5625  81.5625  81.5625]
 [ 87.1875  87.1875  87.1875 ...  87.1875  87.1875  87.1875]]
longitudes = 
 [[  0.      5.625  11.25  ... 343.125 348.75  354.375]
 [  0.      5.625  11.25  ... 343.125 348.75  354.375]
 [  0.      5.625  11.25  ... 343.125 348.75  354.375]
 ...
 [  0.      5.625  11.25  ... 343.125 348.75  354.375]
 [  0.      5.625  11.25  ... 343.125 348.75  354.375]
 [  0.      5.625  11.25  ... 343.125 348.75  354.375]]


In [65]:
import xarray as xr

wb1_dir='/run/media/adamh/HOME'

def xr_open(path): 
    return xr.open_mfdataset(wb1_dir+'/'+path+'/*.nc', combine='by_coords')

constants=xr_open('constants')
_temperature=xr_open('2m_temperature')
temperature=xr_open('temperature')
temperature_850=xr_open('temperature_850')
_u_component_of_wind=xr_open('10m_u_component_of_wind')
_v_component_of_wind=xr_open('10m_v_component_of_wind')
u_component_of_wind=xr_open('u_component_of_wind')
v_component_of_wind=xr_open('v_component_of_wind')
geopotential=xr_open('geopotential')
geopotential_500=xr_open('geopotential_500')
relative_humidity=xr_open('relative_humidity')
specific_humidity=xr_open('specific_humidity')
potential_vorticity=xr_open('potential_vorticity')
vorticity=xr_open('vorticity')
toa_incident_solar_radiation=xr_open('toa_incident_solar_radiation')
total_cloud_cover=xr_open('total_cloud_cover')
total_precipitation=xr_open('total_precipitation')

In [66]:
tv = '2014-01-01T00:00:00.000000000'  # Example time value

my_state['eastward_wind'].values = u_component_of_wind.sel(time=tv).to_array()[0]
my_state['northward_wind'].values = v_component_of_wind.sel(time=tv).to_array()[0]
my_state['air_temperature'].values = temperature.sel(time=tv).to_array()[0]
my_state['specific_humidity'].values = specific_humidity.sel(time=tv).to_array()[0]

#my_state['eastward_wind'].values[:] = np.random.randn(
#   *my_state['eastward_wind'].shape)
#my_state['northward_wind'].values[:] = np.random.randn(
#    *my_state['northward_wind'].shape)
#my_state['air_temperature'].values[:] = 290
#my_state['specific_humidity'].values[:] = 0.0002 

In [67]:
for i in range(1*24*6):
    diag, my_state = dycore(my_state, model_time_step)
    my_state.update(diag)
    my_state['time'] += model_time_step

    if i % (3*6) == 0:
        netcdf_monitor.store(my_state)
        monitor.store(my_state)
        #print('max. zonal wind: ',
        #      np.amax(my_state['eastward_wind'].values))
        #print('max. meridional wind: ',
        #        np.amax(my_state['northward_wind'].values))
        #print('max. humidity: ',
        #      np.amax(my_state['specific_humidity'].values))
        #print('max. temp: ',
        #      my_state['air_temperature'].max(keep_attrs=True).values)

    #print(my_state['time'])

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatched elements: 2048 / 2048 (100%)
Max absolute difference: 172.94808712
Max relative difference: 1.98449885
 x: array([[ 85.760587,  85.760587,  85.760587, ...,  85.760587,  85.760587,
         85.760587],
       [ 80.268779,  80.268779,  80.268779, ...,  80.268779,  80.268779,...
 y: array([[-87.1875, -87.1875, -87.1875, ..., -87.1875, -87.1875, -87.1875],
       [-81.5625, -81.5625, -81.5625, ..., -81.5625, -81.5625, -81.5625],
       [-75.9375, -75.9375, -75.9375, ..., -75.9375, -75.9375, -75.9375],...